In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15000 non-null  int64  
 1   N_Days         15000 non-null  float64
 2   Drug           8459 non-null   object 
 3   Age            15000 non-null  float64
 4   Sex            15000 non-null  object 
 5   Ascites        8460 non-null   object 
 6   Hepatomegaly   8457 non-null   object 
 7   Spiders        8448 non-null   object 
 8   Edema          15000 non-null  object 
 9   Bilirubin      15000 non-null  float64
 10  Cholesterol    6625 non-null   float64
 11  Albumin        15000 non-null  float64
 12  Copper         8343 non-null   float64
 13  Alk_Phos       8453 non-null   float64
 14  SGOT           8449 non-null   float64
 15  Tryglicerides  6578 non-null   float64
 16  Platelets      14418 non-null  float64
 17  Prothrombin    14981 non-null  float64
 18  Stage 

In [4]:
train.isnull().sum()

id                  0
N_Days              0
Drug             6541
Age                 0
Sex                 0
Ascites          6540
Hepatomegaly     6543
Spiders          6552
Edema               0
Bilirubin           0
Cholesterol      8375
Albumin             0
Copper           6657
Alk_Phos         6547
SGOT             6551
Tryglicerides    8422
Platelets         582
Prothrombin        19
Stage               0
Status              0
dtype: int64

In [5]:
test.isnull().sum()

id                  0
N_Days              0
Drug             4300
Age                 0
Sex                 0
Ascites          4296
Hepatomegaly     4300
Spiders          4303
Edema               0
Bilirubin           0
Cholesterol      5558
Albumin             0
Copper           4372
Alk_Phos         4304
SGOT             4304
Tryglicerides    5578
Platelets         357
Prothrombin        14
Stage               0
dtype: int64

In [6]:
cols_to_drop = ['Drug', 'Ascites', 'Hepatomegaly', 'Spiders', 
                'Cholesterol', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides']
train.drop(columns=cols_to_drop, inplace=True, errors='ignore')

train['Platelets'] = train['Platelets'].fillna(train['Platelets'].mean())
train['Prothrombin'] = train['Prothrombin'].fillna(train['Prothrombin'].mean())

In [7]:
train.isnull().sum()

id             0
N_Days         0
Age            0
Sex            0
Edema          0
Bilirubin      0
Albumin        0
Platelets      0
Prothrombin    0
Stage          0
Status         0
dtype: int64

In [8]:
train_ids = train['id']
train.drop(columns=['id'], inplace=True)

In [9]:
X = train.drop(columns=['Status'])
y = train['Status']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [11]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [12]:
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])


In [13]:
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [14]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
])

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(
    n_estimators=600,           # Increased for better learning
    learning_rate=0.02,         # Lower learning rate for better generalization
    max_depth=4,                # Reduced depth to prevent overfitting
    min_samples_split=42,       # Regularization
    min_samples_leaf=18,         # Regularization
    max_features=0.7,           # Feature sampling
    subsample=0.7,              # Instance sampling
    random_state=42
)

In [16]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

In [17]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [18]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['N_Days', 'Age', 'Bilirubin', 'Albumin', 'Platelets', 'Prothrombin',
       'Stage'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Sex', 'Edema'], dtype='object'))])),
                ('classifier',
                 GradientBoostingClassifier(learning_rate=0.02, max_depth=4,
                                            max_features=0.7,
                                            min_samples_leaf=18,
                                            min_samples_split=42,
                                            n_estimators=600, random_state=42,
                                            subsample=0.7))])

In [19]:
y_pred_proba = pipeline.predict_proba(X_test)
loss = log_loss(y_test, y_pred_proba)
print(f"\n✅ Log Loss on validation set: {loss:.5f}")


✅ Log Loss on validation set: 0.38161


In [20]:
test.drop(columns=cols_to_drop, inplace=True, errors='ignore')
test['Platelets'] = test['Platelets'].fillna(test['Platelets'].mean())
test['Prothrombin'] = test['Prothrombin'].fillna(test['Prothrombin'].mean())

In [21]:
test_ids = test['id']
test.drop(columns=['id'], inplace=True)

In [22]:
probs = pipeline.predict_proba(test)
class_names = le.classes_  # use label encoder mapping
submission = pd.DataFrame(probs, columns=[f"Status_{cls}" for cls in class_names])

In [23]:
submission.insert(0, 'id', test_ids)

In [24]:
submission.to_csv("submission3.csv", index=False)
print("\n✅ Submission file created successfully!")
print(submission.head())


✅ Submission file created successfully!
      id  Status_C  Status_CL  Status_D
0  15000  0.949555   0.004317  0.046128
1  15001  0.980808   0.003278  0.015913
2  15002  0.926252   0.008062  0.065686
3  15003  0.060762   0.153459  0.785780
4  15004  0.979158   0.003464  0.017378
